In [1]:
!pip install pinecone

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 4.5 MB/s eta 0:00:00


In [2]:
!pip install pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.5/215.5 kB 3.3 MB/s eta 0:00:00


In [3]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 12.0 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [4]:
!pip install langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.3 MB/s eta 0:00:00


In [5]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 3.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [6]:
!pip install CTransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 53.5 MB/s eta 0:00:00


In [7]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.8 MB/s eta 0:00:00


In [8]:
!pip install pinecone-client

In [9]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyMuPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [10]:
!pip install pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 51.9 MB/s eta 0:00:00


# Load data

In [11]:
def load_pdf(pdf_file_path):
    loader = PyMuPDFLoader(pdf_file_path)
    document = loader.load()
    return document

extracted_data=load_pdf("pocket-medicine_-the-massachusetts-general-hospital-handbook-of-internal-medicine.pdf")

# Creating data to upload at pinecone database

In [12]:
from pinecone import Pinecone

pc = Pinecone(api_key="f1b85353-d127-402d-ac3d-1604f116d6b8")
index = pc.Index("medical")


In [13]:
# Creaate text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [14]:
text_chunks = text_split(extracted_data)
print("length of text_chunks", len(text_chunks))

length of text_chunks 1971


In [15]:
# download embedding model
def download_hugging_face_embedding():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [16]:
embeddings = download_hugging_face_embedding()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
query_result = embeddings.embed_query("Hello World")
print("Length",len(query_result))

Length 384


In [18]:
query_result[0:5]

[-0.034477315843105316,
 0.031023172661662102,
 0.006734910886734724,
 0.02610892429947853,
 -0.03936195746064186]

In [63]:
embedded_data = [[str(i), embeddings.embed_query(str(i))] for i in clean_info]

In [64]:
embedded_data_copy = embedded_data.copy()

In [ ]:
length_check=[i for i in clean_info if i<512]

In [21]:

def truncate_string(string, max_length):
    """
    Truncates a string to a maximum length without losing any meaningful information.
    """
    if len(string) <= max_length:
        return string

    # Split the string into words
    words = string.split()

    # Initialize variables
    truncated_string = ""
    current_length = 0

    # Iterate through words and add them to truncated_string until max_length is reached
    for word in words:
        if current_length + len(word) + 1 <= max_length:  # Add 1 for space
            truncated_string += word + " "
            current_length += len(word) + 1
        else:
            break

    return truncated_string.strip()

In [54]:
important_info = [doc.page_content.split('\nmetadata=')[0] for doc in text_chunks]

In [57]:
important_info[40]

'Normal\nnb, absent in LBBB) followed by LV & RV free wall, with LV\ndominating (nb, RV depol. later and visible in RBBB).\nRBBB\n1. QRS \x07120 msec (110–119 \x08 incomplete)\n2. rSR’ in R precordial leads (V1,V2)\n3. Wide S wave in I and V6\n4.\n\t STT or TWI in R precordial leads\nLBBB\n1. QRS \x07120 msec (110–119 \x08 incomplete)\n2. Broad, slurred, monophasic R in I, aVL,V5–V6 (\t RS in \nV5–V6 if cardiomegaly)\n3. Absence of Q in I,V5, and V6 (may have narrow q in aVL)'

In [58]:
import re
clean_info = [re.sub(r'[^\x00-\x7F]+', '', text) for text in important_info]

In [62]:
len(clean_info[50])

479

In [53]:
text_chunks[40:45]

[Document(page_content='Normal\nnb, absent in LBBB) followed by LV & RV free wall, with LV\ndominating (nb, RV depol. later and visible in RBBB).\nRBBB\n1. QRS \x07120 msec (110–119 \x08 incomplete)\n2. rSR’ in R precordial leads (V1,V2)\n3. Wide S wave in I and V6\n4.\n\t STT or TWI in R precordial leads\nLBBB\n1. QRS \x07120 msec (110–119 \x08 incomplete)\n2. Broad, slurred, monophasic R in I, aVL,V5–V6 (\t RS in \nV5–V6 if cardiomegaly)\n3. Absence of Q in I,V5, and V6 (may have narrow q in aVL)', metadata={'source': 'pocket-medicine_-the-massachusetts-general-hospital-handbook-of-internal-medicine.pdf', 'file_path': 'pocket-medicine_-the-massachusetts-general-hospital-handbook-of-internal-medicine.pdf', 'page': 12, 'total_pages': 282, 'format': 'PDF 1.6', 'title': 'Pocket Medicine: The Massachusetts General Hospital Handbook of Internal Medicine', 'author': ' Sabatine', 'subject': '', 'keywords': '', 'creator': '', 'producer': '', 'creationDate': "D:20110607115028+05'30'", 'modDate

In [22]:
embedded_data_copy = [(truncate_string(sublist[0],510),sublist[1]) for sublist in embedded_data]

In [23]:
def remove_non_ascii(data):
    """
    Remove non-ASCII strings from the given dataset.

    Parameters:
    data (list): A list of strings.

    Returns:
    list: A list with non-ASCII strings removed.
    """
    cleaned_data = []
    for string in data:
        cleaned_string = ''.join(char for char in string if ord(char) < 128)
        cleaned_data.append(cleaned_string)
    return cleaned_data


In [24]:
embedded_data_copy = [(remove_non_ascii(sublist[0]),sublist[1]) for sublist in embedded_data]

In [25]:
len(embedded_data_copy)

1971

In [26]:
embedded_data_copy=[(''.join(sublist[0]),sublist[1]) for sublist in embedded_data]

In [27]:
embedded_data_copy = [(truncate_string(sublist[0],510),sublist[1]) for sublist in embedded_data]

In [28]:
len(embedded_data_copy[40][0])

494

In [29]:
embedded_data_copy = [(''.join(sublist[0]),sublist[1]) for sublist in embedded_data]

In [30]:
embedded_data_copy=[(i.encode('ascii', 'ignore').decode(),j) for i, j in embedded_data_copy[40:1040]]

In [31]:
len(embedded_data_copy[40][0])

997

In [32]:
def is_valid_string(s):
    """
    Verify if a string is under 512 words and contains only ASCII characters.

    Parameters:
    s (str): The string to be verified.

    Returns:
    str: The modified string if valid and all characters are ASCII, otherwise False.
    """
    # Check if the string has more than 512 words
    words = s.split()
    if len(words) > 510:
        s = ' '.join(words[:510])  # Truncate to 512 words

    # Check if all characters in the string are ASCII
    if not all(ord(char) < 128 for char in s):
        # Remove non-ASCII characters
        s = ''.join(char for char in s if ord(char) < 128)

    return s  # Return the modified string if valid, otherwise False


In [33]:
embedded_data_copy = [(is_valid_string(sublist[0]),sublist[1]) for sublist in embedded_data]

In [34]:
embedded_data_copy = [(sublist[0].replace('\\',''),sublist[1]) for sublist in embedded_data]

In [35]:
embedded_data_copy[40]

('page_content=\'Normalnnb, absent in LBBB) followed by LV & RV free wall, with LVndominating (nb, RV depol. later and visible in RBBB).nRBBBn1. QRS x07120 msec (110–119 x08 incomplete)n2. rSR’ in R precordial leads (V1,V2)n3. Wide S wave in I and V6n4.nt STT or TWI in R precordial leadsnLBBBn1. QRS x07120 msec (110–119 x08 incomplete)n2. Broad, slurred, monophasic R in I, aVL,V5–V6 (t RS in nV5–V6 if cardiomegaly)n3. Absence of Q in I,V5, and V6 (may have narrow q in aVL)\' metadata={\'source\': \'pocket-medicine_-the-massachusetts-general-hospital-handbook-of-internal-medicine.pdf\', \'file_path\': \'pocket-medicine_-the-massachusetts-general-hospital-handbook-of-internal-medicine.pdf\', \'page\': 12, \'total_pages\': 282, \'format\': \'PDF 1.6\', \'title\': \'Pocket Medicine: The Massachusetts General Hospital Handbook of Internal Medicine\', \'author\': \' Sabatine\', \'subject\': \'\', \'keywords\': \'\', \'creator\': \'\', \'producer\': \'\', \'creationDate\': "D:20110607115028+0

In [65]:
embedded_data_copy[40]

['Normal\nnb, absent in LBBB) followed by LV & RV free wall, with LV\ndominating (nb, RV depol. later and visible in RBBB).\nRBBB\n1. QRS \x07120 msec (110119 \x08 incomplete)\n2. rSR in R precordial leads (V1,V2)\n3. Wide S wave in I and V6\n4.\n\t STT or TWI in R precordial leads\nLBBB\n1. QRS \x07120 msec (110119 \x08 incomplete)\n2. Broad, slurred, monophasic R in I, aVL,V5V6 (\t RS in \nV5V6 if cardiomegaly)\n3. Absence of Q in I,V5, and V6 (may have narrow q in aVL)',
 [-0.026282884180545807,
  -0.05108261853456497,
  -0.03328751400113106,
  -0.010808603838086128,
  -0.02449924126267433,
  0.08777766674757004,
  -0.10601193457841873,
  0.04956674203276634,
  0.0010649861069396138,
  -0.08374835550785065,
  -0.023028144612908363,
  -0.020684916526079178,
  -0.04205568507313728,
  0.01341057475656271,
  0.0005397626664489508,
  -0.028269920498132706,
  -0.04391090199351311,
  -0.1214853823184967,
  -0.057425595819950104,
  0.06320995837450027,
  0.005014223046600819,
  0.0929502397

In [66]:
from pinecone import Pinecone

pc = Pinecone(api_key="f1b85353-d127-402d-ac3d-1604f116d6b8")
index = pc.Index("grey")

index.upsert(
    vectors = [
    {
         "id":str(i),
        "values": j,
    }
    for i,j in embedded_data_copy[40:1040]
],
    namespace= "grey")

{'upserted_count': 1000}

In [38]:
converted_data = [(i+1, b) for i, (_, b) in enumerate(embedded_data_copy)]

In [42]:
len(converted_data)

1971

In [67]:
query_result = embeddings.embed_query("Acute Coronary Syndrome")


In [48]:
query_result[0]

-0.04734167456626892

In [70]:
output=index.query(
    namespace="grey",
    vector=query_result,
    top_k=2,
    include_values=True,

)

In [71]:
print(output)

{'matches': [{'id': 'ACS\n'
                    '1-6\n'
                    'ACUTE CORONARY SYNDROMES\n'
                    'Myocardial ischemia typically due to atherosclerotic '
                    'plaque rupture S coronary thrombosis\n'
                    'Spectrum of Acute Coronary Syndromes\n'
                    'Dx\n'
                    'UA\n'
                    'NSTEMI\n'
                    'STEMI\n'
                    'Coronary thrombosis\n'
                    'Subtotal\n'
                    'Total\n'
                    'History\n'
                    'angina that is new-onset, crescendo, or\n'
                    'angina at rest\n'
                    'at rest; usually \x0530 min\n'
                    'usually \x0730 min\n'
                    'ECG\n'
                    '\t ST depression and/or TWI\n'
                    'ST elevations\n'
                    'Troponin/CK-MB\n'
                    '\x02\n'
                    '\x03\n'
                    '\x03 \x03

In [73]:
valuable_info = [item['id'] for item in output['matches']]
print(valuable_info)

['ACS\n1-6\nACUTE CORONARY SYNDROMES\nMyocardial ischemia typically due to atherosclerotic plaque rupture S coronary thrombosis\nSpectrum of Acute Coronary Syndromes\nDx\nUA\nNSTEMI\nSTEMI\nCoronary thrombosis\nSubtotal\nTotal\nHistory\nangina that is new-onset, crescendo, or\nangina at rest\nat rest; usually \x0530 min\nusually \x0730 min\nECG\n\t ST depression and/or TWI\nST elevations\nTroponin/CK-MB\n\x02\n\x03\n\x03 \x03\nDdx (causes of myocardial ischemia/infarction other than atherosclerotic plaque rupture)', ', COPD, PTX,WPW\nST elevation (STE) (NEJM 2003;349:2128; Circ 2009;119:e241, e262)\n Acute MI (upward convexity \t TWI) or prior MI with persistent STE\n Coronary spasm (Prinzmetals angina; transient STE in a coronary distribution)\n Myopericarditis (diffuse, upward concavity STE; a/w PR T;Tw usually upright)\n HCMP, Takotsubo CMP, ventricular aneurysm, cardiac contusion \n Pulmonary embolism (occ. STE V1V3; typically associated TWI V1V4, RAD, RBBB)\n Repolarization abnorm

['ACS\n1-6\nACUTE CORONARY SYNDROMES\nMyocardial ischemia typically due to atherosclerotic plaque rupture S coronary thrombosis\nSpectrum of Acute Coronary Syndromes\nDx\nUA\nNSTEMI\nSTEMI\nCoronary thrombosis\nSubtotal\nTotal\nHistory\nangina that is new-onset, crescendo, or\nangina at rest\nat rest; usually \x0530 min\nusually \x0730 min\nECG\n\t ST depression and/or TWI\nST elevations\nTroponin/CK-MB\n\x02\n\x03\n\x03 \x03\nDdx (causes of myocardial ischemia/infarction other than atherosclerotic plaque rupture)', ', COPD, PTX,WPW\nST elevation (STE) (NEJM 2003;349:2128; Circ 2009;119:e241, e262)\n Acute MI (upward convexity \t TWI) or prior MI with persistent STE\n Coronary spasm (Prinzmetals angina; transient STE in a coronary distribution)\n Myopericarditis (diffuse, upward concavity STE; a/w PR T;Tw usually upright)\n HCMP, Takotsubo CMP, ventricular aneurysm, cardiac contusion \n Pulmonary embolism (occ. STE V1V3; typically associated TWI V1V4, RAD, RBBB)\n Repolarization abnormalities']


In [ ]:
import itertools
embedded_data_generator = iter(embedded_data)


In [ ]:
for element in embedded_data_generator:
  print(element)

In [ ]:
def chunks(iterable, batch_size=100):
    """A helper function to break an iterable into chunks of size batch_size."""
    it = iter(iterable)
    chunk = tuple(itertools.islice(it, batch_size))
    while chunk:
        yield chunk
        chunk = tuple(itertools.islice(it, batch_size))

In [ ]:
data=chunks(embedded_data_generator)

In [ ]:
list_data = list(data)
print(list_data)

[]


In [ ]:
with pc.Index('medical', pool_threads=30) as index:
    # Send requests in parallel
    async_results = [
        index.upsert(vectors=ids_vectors_chunk, async_req=True)
        for ids_chunk,vector_chunk in chunks(example_data_generator, batch_size=100)
    ]

In [ ]:
text_chunks[800]

Document(page_content='ond antibody.\nThe latest generation of ELISA tests are 99.5% sen-\nsitive to HIV. Occasionally, the ELISA test will be posi-\ntive for a patient without symptoms of AIDS from a low-\nrisk group. Because this result is likely to be a false-posi-\ntive, the ELISA must be repeated on the same sample of\nthe patient’s blood. If the second ELISA is positive, the\nresult should be confirmed by the Western blot test.\nWESTERN BLOT (IMMUNOBLOT). The Western blot', metadata={'source': 'Medical_book.pdf', 'file_path': 'Medical_book.pdf', 'page': 95, 'total_pages': 637, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creationDate': "D:20041218170002-05'00'", 'modDate': "D:20041218161531-06'00'", 'trapped': ''})

In [ ]:
index.query(
    namespace="( Default )",
    vector=[0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3],
    top_k=2,
    include_values=True,
    include_metadata=True,
    filter={"genre": {"$eq": "action"}}
)